In [13]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


## Librerías

In [14]:
import numpy as np

In [15]:
import tensorflow as tf
from transformers import TFAutoModelForQuestionAnswering
from transformers import pipeline

## Ejemplo Roberta

In [16]:
from transformers import AutoTokenizer

model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [52]:
contexto = 'Es hora de mirar la respuesta a las preguntas. Esta tarea viene en muchos enfoques, pero en el que nos centraremos en esta sección se llama respuesta extractiva de preguntas. Esto implica plantear preguntas sobre un documento e identificar las respuestas como tramos de texto en el propio documento.'
pregunta = '¿En qué tipo de enfoque se centra el documento?'
answers = 'respuesta extractiva de preguntas'
start_char  = 140

In [53]:
inputs = tokenizer(
    pregunta, 
    contexto,
    max_length=90,
    truncation="only_second",
    stride=50,
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
)
inputs.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [41]:
inputs['overflow_to_sample_mapping']

[0, 0, 0]

In [54]:
for ids in inputs["input_ids"]:
    print(tokenizer.decode(ids))

<s>¿En qué tipo de enfoque se centra el documento?</s></s>Es hora de mirar la respuesta a las preguntas. Esta tarea viene en muchos enfoques, pero en el que nos centraremos en esta sección se llama respuesta extractiva de preguntas. Esto implica plantear preg</s>
<s>¿En qué tipo de enfoque se centra el documento?</s></s>a tarea viene en muchos enfoques, pero en el que nos centraremos en esta sección se llama respuesta extractiva de preguntas. Esto implica plantear preguntas sobre un documento e identificar las respuestas como</s>
<s>¿En qué tipo de enfoque se centra el documento?</s></s> centraremos en esta sección se llama respuesta extractiva de preguntas. Esto implica plantear preguntas sobre un documento e identificar las respuestas como tramos de texto en el propio documento.</s>


In [55]:
print('Número de particiones del contexto:', str(len(inputs['input_ids'])))
print('Longitud del atributo offset_mapping', str(len(inputs['offset_mapping'])))
print('Longitud de caracteres de la primera partición', str(len('Es hora de mirar la respuesta a las preguntas. Esta tarea viene en muchos enfoques, pero en el que nos centraremos en esta sección se llama respuesta extractiva de preguntas. Esto implica plantear preg')))
inputs['offset_mapping'][0]

Número de particiones del contexto: 3
Longitud del atributo offset_mapping 3
Longitud de caracteres de la primera partición 201


[(0, 0),
 (0, 1),
 (0, 1),
 (1, 3),
 (4, 6),
 (6, 7),
 (8, 11),
 (11, 12),
 (13, 15),
 (16, 18),
 (18, 20),
 (20, 23),
 (24, 26),
 (27, 31),
 (31, 33),
 (34, 36),
 (37, 45),
 (45, 46),
 (46, 47),
 (0, 0),
 (0, 0),
 (0, 2),
 (3, 6),
 (6, 7),
 (8, 10),
 (11, 14),
 (14, 16),
 (17, 19),
 (20, 24),
 (24, 25),
 (25, 29),
 (30, 31),
 (32, 35),
 (36, 39),
 (39, 40),
 (40, 43),
 (43, 45),
 (45, 46),
 (47, 50),
 (50, 51),
 (52, 53),
 (53, 57),
 (58, 60),
 (60, 63),
 (64, 66),
 (67, 71),
 (71, 73),
 (74, 76),
 (76, 78),
 (78, 82),
 (82, 83),
 (84, 87),
 (87, 88),
 (89, 91),
 (92, 94),
 (95, 98),
 (99, 102),
 (103, 107),
 (107, 109),
 (109, 112),
 (112, 114),
 (115, 117),
 (118, 121),
 (121, 122),
 (123, 125),
 (125, 128),
 (128, 130),
 (131, 133),
 (134, 136),
 (136, 139),
 (140, 144),
 (144, 145),
 (145, 149),
 (150, 157),
 (157, 160),
 (161, 163),
 (164, 167),
 (167, 168),
 (168, 171),
 (171, 173),
 (173, 174),
 (175, 178),
 (178, 179),
 (180, 184),
 (184, 187),
 (188, 193),
 (193, 196),
 (197,

In [44]:
offset_mapping = inputs.pop("offset_mapping")
sample_map = inputs.pop("overflow_to_sample_mapping")
start_positions = []
end_positions = []

In [ ]:
    for i, offset in enumerate(offset_mapping):
        
        sample_idx = sample_map[i]
        answer = answers
        end_char = start_char + len(answers)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    

In [66]:
i = 0
offset = offset_mapping[0]   
sample_idx = sample_map[i]
answer = answers
end_char = start_char + len(answers)
sequence_ids = inputs.sequence_ids(i)

print('offset: ', offset_mapping)
print('sample_idx', sample_idx)
print('end_char:', end_char)
print('sequence_ids:', sequence_ids )

offset:  [[(0, 0), (0, 1), (0, 1), (1, 3), (4, 6), (6, 7), (8, 11), (11, 12), (13, 15), (16, 18), (18, 20), (20, 23), (24, 26), (27, 31), (31, 33), (34, 36), (37, 45), (45, 46), (46, 47), (0, 0), (0, 0), (0, 2), (3, 6), (6, 7), (8, 10), (11, 14), (14, 16), (17, 19), (20, 24), (24, 25), (25, 29), (30, 31), (32, 35), (36, 39), (39, 40), (40, 43), (43, 45), (45, 46), (47, 50), (50, 51), (52, 53), (53, 57), (58, 60), (60, 63), (64, 66), (67, 71), (71, 73), (74, 76), (76, 78), (78, 82), (82, 83), (84, 87), (87, 88), (89, 91), (92, 94), (95, 98), (99, 102), (103, 107), (107, 109), (109, 112), (112, 114), (115, 117), (118, 121), (121, 122), (123, 125), (125, 128), (128, 130), (131, 133), (134, 136), (136, 139), (140, 144), (144, 145), (145, 149), (150, 157), (157, 160), (161, 163), (164, 167), (167, 168), (168, 171), (171, 173), (173, 174), (175, 178), (178, 179), (180, 184), (184, 187), (188, 193), (193, 196), (197, 200), (200, 201), (0, 0)], [(0, 0), (0, 1), (0, 1), (1, 3), (4, 6), (6, 7), 

In [74]:
inputs["start_positions"]

[70, 52, 34]

In [75]:
inputs["end_positions"] 

[79, 61, 43]

# Ejemplo T5


In [76]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")     

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [80]:
question_plus = f"answer_me: {str(pregunta)}"
question_plus += f" context: {str(contexto)} </s>"
question_plus

'answer_me: ¿En qué tipo de enfoque se centra el documento? context: Es hora de mirar la respuesta a las preguntas. Esta tarea viene en muchos enfoques, pero en el que nos centraremos en esta sección se llama respuesta extractiva de preguntas. Esto implica plantear preguntas sobre un documento e identificar las respuestas como tramos de texto en el propio documento. </s>'

In [81]:
answer_plus = f"{answers} </s>"
answer_plus

'respuesta extractiva de preguntas </s>'

In [83]:
encoder_inputs = tokenizer(question_plus, truncation=True, 
                               return_tensors='tf', max_length=90,
                              pad_to_max_length=True)
    
decoder_inputs = tokenizer(answer_plus, truncation=True, 
                               return_tensors='tf', max_length=10,
                              pad_to_max_length=True)
    
input_ids = encoder_inputs['input_ids'][0]
input_attention = encoder_inputs['attention_mask'][0]
target_ids = decoder_inputs['input_ids'][0]
target_attention = decoder_inputs['attention_mask'][0]
    
outputs = {'input_ids':input_ids, 'attention_mask': input_attention, 
               'labels':target_ids, 'decoder_attention_mask':target_attention}
outputs

{'input_ids': <tf.Tensor: shape=(90,), dtype=int32, numpy=
 array([ 1525,   834,   526,    10,     3,     2,  8532,     3,  7195,
         2226,    32,    20,     3,    35,    89,    32,   835,   142,
        21551,     9,     3,    15,    40,  1708,    32,    58,  2625,
           10,  1122,     3, 21783,    20,  1870,   291,    50, 23259,
           76,   222,     9,     3,     9,    50,     7,   554,   122,
        14016,     7,     5,  7545,     9,     3,    17,   498,  2240,
           29,    15,     3,    35,   231,    32,     7,     3,    35,
           89,    32,  7771,     6,   399,    32,     3,    35,     3,
           15,    40,   238,  2844, 21551,   355,  3972,     3,    35,
          259,     9,  4220, 12765,   142,     3,   195,   265,     1],
       dtype=int32)>,
 'attention_mask': <tf.Tensor: shape=(90,), dtype=int32, numpy=
 array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,